In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
with open("small_shakespeare.txt", "r", encoding="utf-8") as file:
    text = file.read().lower()


In [4]:

chars = sorted(list(set(text)))
vocab_size = len(chars)

char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for i, c in enumerate(chars)}

print("Vocabulary size:", vocab_size)


Vocabulary size: 26


In [5]:

encoded_text = np.array([char_to_idx[c] for c in text])


In [6]:

seq_length = 100
X = []
y = []

for i in range(0, len(encoded_text) - seq_length):
    X.append(encoded_text[i:i + seq_length])
    y.append(encoded_text[i + seq_length])

X = np.array(X)
y = to_categorical(y, num_classes=vocab_size)

print("Input shape:", X.shape)
print("Output shape:", y.shape)


Input shape: (111, 100)
Output shape: (111, 26)


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [8]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=seq_length),
    LSTM(256, return_sequences=True),
    Dropout(0.2),
    LSTM(256),
    Dense(vocab_size, activation="softmax")
])

model.compile(
    loss="categorical_crossentropy",
    optimizer="adam"
)

model.summary()


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
early_stop = EarlyStopping(monitor="val_loss", patience=3)
checkpoint = ModelCheckpoint(
    "lstm_text_gen.h5",
    monitor="val_loss",
    save_best_only=True
)

history = model.fit(
    X, y,
    batch_size=64,
    epochs=10,              
    validation_split=0.1
)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 863ms/step - loss: 2.9394 - val_loss: 3.1609
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 702ms/step - loss: 2.9395 - val_loss: 3.1576
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 803ms/step - loss: 2.9441 - val_loss: 3.1491
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 689ms/step - loss: 2.9267 - val_loss: 3.1512
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 749ms/step - loss: 2.9219 - val_loss: 3.1697
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 688ms/step - loss: 2.9286 - val_loss: 3.1584
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 758ms/step - loss: 2.9183 - val_loss: 3.1252
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 729ms/step - loss: 2.8995 - val_loss: 3.1151
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 723ms/step - loss: 2.8841 - val_loss: 3.1220
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 783ms/step - loss: 2.8757 - val_loss: 3.1271


In [17]:
def generate_text(model, seed_text, length=200, temperature=0.4):
    seed_text = seed_text.lower()
    seed_text = " " * (seq_length - len(seed_text)) + seed_text

    generated = seed_text

    for _ in range(length):
        seq = [char_to_idx.get(c, 0) for c in generated[-seq_length:]]
        seq = pad_sequences([seq], maxlen=seq_length)

        preds = model.predict(seq, verbose=0)[0]
        preds = np.log(preds + 1e-8) / temperature
        preds = np.exp(preds) / np.sum(np.exp(preds))

        next_idx = np.random.choice(len(preds), p=preds)
        generated += idx_to_char[next_idx]

    return generated.strip()


In [18]:
print(generate_text(model, "to be", length=200,temperature=0.2))


to bee eee el              e      e   eeeee   l         e        ee e e   e          l   eee ee  e   o f      e            o    l        e  e  e   ee ee  l  ee  e e       e    e ee         nee  eo   a
